In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

In [7]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini", temperature=0)

## Agent

Consists of two components:
* AgentWorker - Responsible for executing the next step of a given agent
* AgentRunner - Overall task dispatcher, responsible for creating a task, orchestrates runs of AgentWorkers on top of tasks


In [8]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [9]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "agent roles in MetaGPT"}
=== Function Output ===
In MetaGPT, the agent roles include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities and skills tailored to their tasks within the software company simulated by MetaGPT. The agents are specialized in their roles to efficiently collaborate and contribute to solving complex tasks or problems.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "communication between agents in MetaGPT"}
=== Function Output ===
Agents in MetaGPT communicate through a structured communication protocol that involves a shared message pool. This pool allows agents to publish and access structured messages, enabling efficient exchange of information among different role

In [10]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 9
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2025-01-30
last_modified_date: 2025-01-30

Preprint
Table 2: Comparison of capabilities for MetaGPT and other approaches. ‘!’ indicates the
presence of a specific feature in the corresponding framework, ‘%’ its absence.
Framework Capabiliy AutoGPT LangChain AgentVerse ChatDev MetaGPT
PRD generation % % % % !
Tenical design genenration % % % % !
API interface generation % % % % !
Code generation ! ! ! ! !
Precompilation execution % % % % !
Role-based task management % % % ! !
Code review % % ! ! !
Table 3: Ablation study on roles. ‘#’ denotes ‘The number of’, ‘Product’ denotes ‘Product man-
ager’, and ‘Project’ denotes ‘Project manager’. ‘ !’ indicates the addition of a specific role. ‘Revi-
sions’ refers to ‘Human Revision Cost’.
Engineer Product Architect Project #Agents #Lines Expense Revisions Executability
! % % % 1 83.0 $ 0.915 10 1.0
! ! % % 2 112.0 $ 1.059 6.

## Full Agent Reasoning Loop

* agent.query allows you to query the agent in a one off manner, but does not preserve state.
* agent is able to maintain chats in a conversational mmemmory buffer.
* memory module can be customized but by default is a flat list of items, that's a rolling buffer depending on context window of LLM

In [12]:
response = agent.chat(
    "Tell me about the evaluation datasets used. There might be multiple"
)

Added user message to memory: Tell me about the evaluation datasets used. There might be multiple
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "evaluation datasets in MetaGPT"}
=== Function Output ===
SoftwareDev dataset
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "evaluation datasets used in MetaGPT"}
=== Function Output ===
SoftwareDev dataset
=== LLM Response ===
The primary evaluation dataset used in MetaGPT is the **SoftwareDev dataset**, which includes 70 diverse software development tasks. There is no mention of multiple datasets in the available information, indicating that the SoftwareDev dataset is the main focus for evaluation purposes. If there are additional datasets, they are not specified in the current context.


In [13]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results on SoftwareDev dataset in MetaGPT"}
=== Function Output ===
MetaGPT achieves a score of 3.75 for executability on the SoftwareDev dataset, outperforming ChatDev in various metrics such as running times, token usage, lines of code per file, total code lines, productivity, and human revision cost.
=== LLM Response ===
MetaGPT achieves a score of **3.75 for executability** on the SoftwareDev dataset. It outperforms ChatDev across various metrics, including:

- Running times
- Token usage
- Lines of code per file
- Total code lines
- Productivity
- Human revision cost

These results indicate that MetaGPT is more efficient and effective in handling software development tasks compared to ChatDev.


## Agent Control
* **Decoupling Task Creation and Execution**: Users gain the flexibility to schedule task execution according to their needs.
* **Enhanced Debuggability**: Offers deeper insights into each step of the execution process, improved troubleshooting capabilities.
* **Steerability**: Allows users to directly modify intermediate steps and incorporate human feedback for refined control

In [17]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [18]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [19]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "agent roles in MetaGPT"}
=== Function Output ===
Engineer, Product Manager, Architect, Project Manager, and QA Engineer are the roles specified in MetaGPT.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "communication between agents in MetaGPT"}
=== Function Output ===
Agents in MetaGPT communicate through a structured communication protocol that involves a shared message pool. This pool allows agents to publish structured messages and subscribe to relevant messages based on their profiles. By utilizing a publish-subscribe mechanism, agents can efficiently exchange information and access necessary data from other roles in the system. This approach enhances communication efficiency by enabling transparent message exchange and eliminating the need for

In [20]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 0f9f08fa-0cf2-4c8a-9c39-17126f278f7f: 1
Engineer, Product Manager, Architect, Project Manager, and QA Engineer are the roles specified in MetaGPT.


In [21]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 0f9f08fa-0cf2-4c8a-9c39-17126f278f7f: 1


TaskStep(task_id='0f9f08fa-0cf2-4c8a-9c39-17126f278f7f', step_id='76a8ac5e-fca7-4a59-8145-c26a58c4b7fe', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [22]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information by using a shared message pool to publish structured messages. They can also subscribe to relevant messages based on their profiles. Additionally, agents monitor the environment (i.e., the message pool) to identify important observations, such as messages from other agents, which can directly trigger actions or assist in completing tasks.


In [23]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
In MetaGPT, agents share information through a shared message pool where they can publish structured messages. Each agent can subscribe to messages that are relevant to their roles based on their profiles. This setup allows agents to monitor the environment (the message pool) for important observations, such as messages from other agents, which can trigger actions or assist in completing tasks. This mechanism enhances collaboration and ensures that agents have access to the information they need to perform their roles effectively.
True


In [24]:
response = agent.finalize_response(task.task_id)

In [25]:
print(str(response))

In MetaGPT, agents share information through a shared message pool where they can publish structured messages. Each agent can subscribe to messages that are relevant to their roles based on their profiles. This setup allows agents to monitor the environment (the message pool) for important observations, such as messages from other agents, which can trigger actions or assist in completing tasks. This mechanism enhances collaboration and ensures that agents have access to the information they need to perform their roles effectively.
